In [1]:
import folium
from folium import Map, Marker

#import geojson
import json
#import io

from pprint import pprint

from openrouteservice import client
from openrouteservice.directions import directions
#from openrouteservice.isochrones import isochrones

## Rostock construction sites

In [3]:
#import shapely.geometry
#import shapely
#from shapely import geometry, wkt
#from shapely.geometry import Point
#from shapely.geometry import mapping, shape

In [17]:
api_key = '58d904a497c67e00015b45fc2a6b6872037d44119582ef40cdf264c4' # individual api key
clnt = client.Client(key=api_key) # creating client with api key
map2 = folium.Map(tiles='Stamen Toner', location=([54.13207, 12.101612]), zoom_start=12) # creating map

with open('baustellen_rostock.json') as l: # Load in construction data points
    rostock_json = json.load(l)
    
construction_points = []
for poly in rostock_json['features']:
    poly_coords2 = poly['geometry']['coordinates']
    folium.features.Marker(list(reversed(poly_coords2)), popup='Construction').add_to(map2)
    construction_points.append(poly_coords2)
    
map2

In [6]:
# Create polygons arpund construction points
t = 0.0001
construction_areas = []
for x,y in construction_points:
    point_area = [[x-t, y-t], [x+t, y-t], [x+t, y+t], [x-t, y+t], [x-t, y-t]]
    construction_areas.append(point_area)

In [7]:
# Data is needed as strings in listed lists
coords_constr = []
poly_coords_list = []
multipoly_list = []
for poly in construction_areas:
    for row in poly:
        coords = list(row)
        i = 0
        for l in coords:
            i = i+1
            if i%2 != 0:
                x = l
            if i%2 == 0:
                y = l
                
                kombi_coords = [str(x), str(y)]
                poly_coords_list.append(kombi_coords)
    
    multipoly_list.append([poly_coords_list])
    poly_coords_list = []
pprint(multipoly_list)       

[[[['12.15740204031749', '54.09691611522543'],
   ['12.157602040317489', '54.09691611522543'],
   ['12.157602040317489', '54.09711611522544'],
   ['12.15740204031749', '54.09711611522544'],
   ['12.15740204031749', '54.09691611522543']]],
 [[['12.11544626899832', '54.08890568600825'],
   ['12.115646268998319', '54.08890568600825'],
   ['12.115646268998319', '54.08910568600826'],
   ['12.11544626899832', '54.08910568600826'],
   ['12.11544626899832', '54.08890568600825']]],
 [[['12.043240942579606', '54.152691808058755'],
   ['12.043440942579606', '54.152691808058755'],
   ['12.043440942579606', '54.15289180805876'],
   ['12.043240942579606', '54.15289180805876'],
   ['12.043240942579606', '54.152691808058755']]],
 [[['12.123406538015114', '54.08161839942496'],
   ['12.123606538015114', '54.08161839942496'],
   ['12.123606538015114', '54.08181839942497'],
   ['12.123406538015114', '54.08181839942497'],
   ['12.123406538015114', '54.08161839942496']]],
 [[['12.137959957633909', '54.08532

In [10]:
# Variable with just three construction sites
test_coords = [[[['12.15650204031749', '54.09601611522544'],
   ['12.158502040317488', '54.09601611522544'],
   ['12.158502040317488', '54.098016115225434'],
   ['12.15650204031749', '54.098016115225434'],
   ['12.15650204031749', '54.09601611522544']]],
 [[['12.11454626899832', '54.08800568600826'],
   ['12.116546268998318', '54.08800568600826'],
   ['12.116546268998318', '54.090005686008254'],
   ['12.11454626899832', '54.090005686008254'],
   ['12.11454626899832', '54.08800568600826']]],
 [[['12.042340942579607', '54.15179180805876'],
   ['12.044340942579606', '54.15179180805876'],
   ['12.044340942579606', '54.153791808058756'],
   ['12.042340942579607', '54.153791808058756'],
   ['12.042340942579607', '54.15179180805876']]]]

In [11]:
# Request of direction data avoiding construction areas
route_request = {'coordinates': [[12.088019, 54.084592], [12.129314, 54.083027]],
            'format_out': 'geojson',
            'profile': 'driving-car',
            'preference': 'shortest',
            'options': {'avoid_polygons': {'type': 'MultiPolygon', 'coordinates': test_coords}}
             }
route_rostock = clnt.directions(**route_request)
folium.GeoJson(route_rostock).add_to(map2)

folium.Marker([54.084592, 12.088019], popup='Start', icon=folium.Icon(color='green')).add_to(map2) 
folium.Marker([54.083027, 12.129314], popup='Destination', icon=folium.Icon(color='green')).add_to(map2)
map2

In [13]:
# Mark construction sites 
switched_coords = []
for inner in construction_areas:
    coords = [(y,x) for x,y in inner]
    switched_coords.append(coords)

for n in construction_points:
    folium.features.PolygonMarker(switched_coords, color='red', fill_color='red', fill_opacity=0.2).add_to(map2)
map2

In [ ]:
all_points = []
alle_exter = []
for x,y in rostock_polygon:
    alle = Point(x,y).buffer(0.0001)
    #all_points.append(alle)
    #alle_wkt = wkt.loads(alle)
    alle_exter += [list(alle.exterior.coords)]
    #pprint(alle_exter)
#alle_coords += [(y,x) for x,y in alle_exter]

## Berlin Baustellen areas (Gebäude)

In [2]:
api_key = '58d904a497c67e00015b45fc2a6b6872037d44119582ef40cdf264c4' # individual api key
clnt = client.Client(key=api_key) # creating client with api key
map1 = folium.Map(tiles='Stamen Toner', location=([52.52066, 13.407269]), zoom_start=14) # creating map


with open('construction_area.geojson') as f:
    construction_json = json.load(f)
    
construction_polygon = []
poly_coords = []
for poly in construction_json['features']:
    poly_coords = poly['geometry']['coordinates']
    for wer in poly_coords:
        switched = [[y,x] for x,y in wer]
        folium.features.PolygonMarker(switched, color='red', fill_color='red', fill_opacity=0.5).add_to(map1)
    construction_polygon.append(poly_coords)

map1

In [ ]:
# requesting data
#for n in construction_polygon:
route_car = {'coordinates': [[13.337402, 52.531992], [13.528976, 52.529799]],
            'format_out': 'geojson',
            'profile': 'driving-car',
            'preference': 'shortest',
            'options': {'avoid_polygons': {'type': 'Polygon', 'coordinates': n}}
                }
route = clnt.directions(**route_car)
folium.GeoJson(route).add_to(map1)

folium.Marker([52.531992, 13.337402], popup='start').add_to(map1) 
folium.Marker([52.529799, 13.528976], popup='destination').add_to(map1) 
map1